In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
iris = load_iris()

In [3]:
x = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

In [4]:
# 데이터 분할
x_train, x_test, y_train, y_test = q(x, y, test_size = 0.2, random_state=42)

In [5]:
std_scaler = StandardScaler()
std_scaler.fit(x_train)
x_train_tensor = torch.from_numpy(std_scaler.transform(x_train)).float()
x_test_tensor = torch.from_numpy(std_scaler.transform(x_test)).float()
y_train_tensor = torch.from_numpy(y_train).long()
y_test_tensor = torch.from_numpy(y_test).long()

In [9]:
print(x_train_tensor.shape, x_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape)

torch.Size([120, 4]) torch.Size([30, 4]) torch.Size([120]) torch.Size([30])


In [10]:
nb_epochs = 1000 # 에폭 1000회
minibatch_size = 120 

In [20]:
class Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.linear_layers = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.LeakyReLU(0, 1),
            nn.Linear(100,20),
            nn.LeakyReLU(0,1),
            nn.Linear(20,5),
            nn.LeakyReLU(0,1),
            nn.Linear(5, output_dim),
            # nn.Softmax(dim=-1)
            nn.LogSoftmax(dim=-1)
        )

    def forward(self, x):
        y = self.linear_layers(x)
        return y

In [21]:
input_dim = x_train_tensor.size(-1)
output_dim = 3 # iris는 0, 1, 2 multi label에 대한 확률값을 구해야 하므로, output_dim이 3이 된다.
print(input_dim, output_dim)
model = Model(input_dim, output_dim)

# loss_func = nn.CrossEntropyLoss() # softmax는 CrossEntropyLoss() 로 진행해야 한다.
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())

4 3


In [32]:
for index in range(nb_epochs):
    indices = torch.randperm(x_train_tensor.size(0))

    x_batch_list = torch.index_select(x_train_tensor, 0, index = indices) # 섞기
    y_batch_list = torch.index_select(y_train_tensor, 0, index = indices)
    x_batch_list = x_batch_list.split(minibatch_size, 0) # 미니배치 사이즈로 나누기
    y_batch_list = y_batch_list.split(minibatch_size, 0)

    epoch_loss = list()
    for x_minibatch, y_minibatch in zip(x_batch_list, y_batch_list):
        y_minibatch_pred = model(x_minibatch)
        loss = loss_func(y_minibatch_pred, y_minibatch)
        epoch_loss.append(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if(index%100) == 0:
        print(index, sum(epoch_loss) / len(epoch_loss))
        

0 tensor(0.0310, grad_fn=<DivBackward0>)
100 tensor(0.0294, grad_fn=<DivBackward0>)
200 tensor(0.0274, grad_fn=<DivBackward0>)
300 tensor(0.0251, grad_fn=<DivBackward0>)
400 tensor(0.0230, grad_fn=<DivBackward0>)
500 tensor(0.0210, grad_fn=<DivBackward0>)
600 tensor(0.0193, grad_fn=<DivBackward0>)
700 tensor(0.0175, grad_fn=<DivBackward0>)
800 tensor(0.0160, grad_fn=<DivBackward0>)
900 tensor(0.0149, grad_fn=<DivBackward0>)


In [33]:
# 테스트셋 기반 evaluation
model.eval() # 평가모드로 전환
with torch.no_grad():
    y_test_pred = model(x_test_tensor) # 여기서 확률값이 나옴
    y_pred_list = torch.argmax(y_test_pred, dim=1) # 확률값에서 큰거 골라

In [34]:
# mini_batch size 기반 예측
y_pred_list = list()
x_test_batch_list = x_test_tensor.split(minibatch_size, 0)
model.eval()
with torch.no_grad():
    for x_minibatch in x_test_batch_list:
        y_test_pred = model(x_minibatch)
        print(y_test_pred.shape)
        y_test_pred = torch.argmax(y_test_pred, dim=1)
        print(y_test_pred.shape)
        # y_pred_list.extend(y_test_pred.squeeze().detach().tolist()) # 사실 여기서 squeeze를 할 필요는 없음
        y_pred_list.extend(y_test_pred.detach().tolist())
# y_pred_list = torch.tensor(y_pred_list).unsqueeze(1) # 여기서도 unsqueeze를 할 필요는 없다
y_pred_list = torch.tensor(y_pred_list)

torch.Size([30, 3])
torch.Size([30])


In [35]:
print(y_pred_list.shape, y_test_tensor.shape)

torch.Size([30]) torch.Size([30])


In [36]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

print("Confusion Matrix\n", str(confusion_matrix(y_test_tensor, y_pred_list)))
print("Precision List:\t", str( precision_score(y_test_tensor, y_pred_list, average=None) ) )
print("Macro Precision:\t", str( precision_score(y_test_tensor, y_pred_list, average='macro' ) ) )
print ("Macro Precision Formula:", str( sum(precision_score(y_test_tensor, y_pred_list, average=None) ) / 3 )) # macro 계산을 수동으로 만든
print("Micro Precision:\t", str( precision_score(y_test_tensor, y_pred_list, average='micro') ) )

print("Recall List:\t", str( precision_score(y_test_tensor, y_pred_list, average=None) ) )
print("Macro Recall:\t", str( recall_score(y_test_tensor, y_pred_list, average='macro') ) )
print("Micro Recall:\t", str( recall_score(y_test_tensor, y_pred_list, average='micro') ) )

print("Macro F1 Score List:\t", str( f1_score(y_test_tensor, y_pred_list, average=None) ) )
print("Macro F1 Score:\t", str( f1_score(y_test_tensor, y_pred_list, average='macro') ) )
print("Micro F1 Score:\t", str( f1_score(y_test_tensor, y_pred_list, average='micro') ) )

Confusion Matrix
 [[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
Precision List:	 [1. 1. 1.]
Macro Precision:	 1.0
Macro Precision Formula: 1.0
Micro Precision:	 1.0
Recall List:	 [1. 1. 1.]
Macro Recall:	 1.0
Micro Recall:	 1.0
Macro F1 Score List:	 [1. 1. 1.]
Macro F1 Score:	 1.0
Micro F1 Score:	 1.0


In [ ]:
# early stopping
# 과적홥 되는거 같으면 중간에 멈춰주는 역